In [1]:
import pandas as pd
import ast

In [ ]:
movies_df = pd.read_csv('data/movies.csv')
movies_metadata_df = pd.read_csv('data/movies_metadata.csv')

/var/folders/c6/k78nzngn7nbdd0786bxl3s0c0000gn/T/ipykernel_76118/975769827.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv('data/movies_metadata.csv')


In [3]:
movies_df['cast'] = movies_df['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [4]:
actor_movie ={}

In [5]:
for movie in movies_df.itertuples():
    movie_id = movie.tmdbId
    for actor in movie.cast:
        actor_id = actor.get('id')
        if actor_id not in actor_movie:
            actor_movie[actor_id] = set()
        actor_movie[actor_id].add(movie_id)

In [6]:
#drop actors with less than 1 movie
actor_movie = {actor: movies for actor, movies in actor_movie.items() if len(movies) >= 1}

In [7]:
movies_metadata_df['id'] = pd.to_numeric(movies_metadata_df['id'], errors='coerce')
movies_df = movies_df.merge(movies_metadata_df[['id', 'overview']], left_on='tmdbId', right_on='id', how='left')

In [8]:
movies_df.head()

,Unnamed: 0,title,year,bt_score,dubious,imdbid,tmdbId,genres,popularity,production_companies,...,vote_count,cast,crew,budget,cast_gender,crew_gender,cast_female_representation,crew_female_representation,id,overview
0,0,Cinderella,1899,3,0,230,114108,"['Drama', 'Family', 'Fantasy', 'Romance']",3.762,"['Star-Film', 'Georges Méliès']",...,79.0,"[{'adult': False, 'gender': 2, 'id': 11523, 'k...","[{'adult': False, 'gender': 2, 'id': 11523, 'k...",0.0,"[2, 0, 1, 0, 1, 0]","[2, 2, 2, 2]",33.333333,0.000000,114108.0,A fairy godmother magically turns Cinderella's...
1,1,Gretchen the Greenhorn,1916,3,0,6745,126925,[],1.942,[],...,7.0,"[{'adult': False, 'gender': 1, 'id': 30779, 'k...","[{'adult': False, 'gender': 2, 'id': 42060, 'k...",0.0,"[1, 2, 2, 2]","[2, 2, 2]",25.000000,0.000000,126925.0,Gretchen Van Houck is just arriving in the USA...
2,2,Snow White,1916,3,0,7361,174598,"['Fantasy', 'Drama']",0.817,[],...,9.0,"[{'adult': False, 'gender': 0, 'id': 1658947, ...","[{'adult': False, 'gender': 2, 'id': 28968, 'k...",0.0,"[0, 1, 2, 2, 0]",[2],20.000000,0.000000,174598.0,"Snow White, a beautiful girl, is despised by a..."
3,3,The Poor Little Rich Girl,1917,3,0,8443,95866,"['Romance', 'Comedy', 'Drama', 'Fantasy']",2.023,['Artcraft Pictures Corporation'],...,23.0,"[{'adult': False, 'gender': 1, 'id': 100047, '...","[{'adult': False, 'gender': 2, 'id': 13335, 'k...",0.0,"[1, 0, 0, 0, 2, 0, 1, 1, 0, 2, 0, 1, 1]","[2, 2, 1, 1, 2, 2, 0]",38.461538,28.571429,95866.0,"Gwen's family is rich, but her parents ignore ..."
4,4,Stella Maris,1918,3,0,9652,70753,['Drama'],1.310,['Mary Pickford Company'],...,19.0,"[{'adult': False, 'gender': 1, 'id': 100047, '...","[{'adult': False, 'gender': 1, 'id': 34741, 'k...",0.0,"[1, 1, 2, 2, 1, 0, 0, 2]","[1, 2, 2, 0, 2, 2]",37.500000,16.666667,70753.0,"Stella Maris is a beautiful, crippled girl, wh..."


In [14]:
att = ['title', 'year', 'bt_score', 'genres', 'popularity', 'production_companies',
    'production_countries', 'release_date', 'revenue', 'vote_average', 'vote_count', 'budget', 'cast_gender'
    , 'cast_female_representation', 'overview']

In [15]:
#create a graph where nodes are movies that share actors
import networkx as nx
from itertools import combinations
G = nx.Graph()
#add nodes
for movie in movies_df.itertuples():
    G.add_node(movie.tmdbId)
    for attribute in att:
        G.nodes[movie.tmdbId][attribute] = getattr(movie, attribute)
    

#add edges
for actor, movies in actor_movie.items():
    for movie1, movie2 in combinations(movies, 2):
        #add attribute
        if not G.has_edge(movie1, movie2):
            G.add_edge(movie1, movie2)
            G[movie1][movie2]['weight'] = 1
        else:
            G[movie1][movie2]['weight'] += 1

print(f"Number of nodes in the graph: {G.number_of_nodes()}")
print(f"Number of edges in the graph: {G.number_of_edges()}")

Number of nodes in the graph: 7271
Number of edges in the graph: 375832


In [16]:
#select one edge with highest weight
max_edge = max(G.edges(data=True), key=lambda x: x[2]['weight'])
print(f"Edge with highest weight: {max_edge[0]} - {max_edge[1]} with weight {max_edge[2]['weight']}")

Edge with highest weight: 12444 - 12445 with weight 55


In [17]:
G.nodes[max_edge[1]]

{'title': 'Harry Potter and the Deathly Hallows: Part 2',
 'year': 2011,
 'bt_score': 3,
 'genres': "['Fantasy', 'Adventure']",
 'popularity': 175.257,
 'production_companies': "['Warner Bros. Pictures', 'Heyday Films']",
 'production_countries': "['United Kingdom', 'United States of America']",
 'release_date': '07/07/2011',
 'revenue': 1341511219.0,
 'vote_average': 8.1,
 'vote_count': 17299.0,
 'budget': 125000000.0,
 'cast_gender': '[2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 0, 0, 2, 0, 0, 2, 1, 2, 2, 1, 1, 0, 1, 1, 2, 2, 1, 2, 0, 2, 0, 2, 1, 2, 0, 0, 0, 2, 2, 2, 2, 0, 0, 1, 1, 1, 0, 0, 2, 0, 2, 1, 2, 0, 2, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 2, 1, 0, 2, 2, 0, 2, 2, 2, 0, 1, 2]',
 'cast_female_representation': 27.0,
 'overview': "Harry, Ron and Hermione continue their quest to vanquish the evil Voldemort once and for all. Just as things begin to look hopeless for the young wizards, Harry discovers a trio of magical objects that end

In [21]:
#highest degree centrality
degree_centrality = nx.degree_centrality(G)
max_degree_node = max(degree_centrality, key=degree_centrality.get)
print(f"Node with highest degree centrality: {G.nodes[max_degree_node]['title']} with centrality {degree_centrality[max_degree_node]}")

Node with highest degree centrality: The Player with centrality 0.09202200825309491
